In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
import fasttext
import fasttext.util

In [2]:
input_data = []

with open('./sts-2017-en-es/En_Es_STS/STS.input.en-es.train.txt', 'r') as inFile:
    input_data = inFile.readlines()

X = []
for data in input_data:
    X.append(data.split('\t')[:2])

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

def sentence_preprocessor(data, language):
    cleaned_data = []
    stop_words = set(stopwords.words(language))
    for x in data:
        for c in string.punctuation:
            if c in x:
                x = x.replace(c,'')
        tokenized_x = word_tokenize(x)
        tokenized_x = [word for word in tokenized_x if word not in stop_words]

        cleaned_data.append(tokenized_x)
    return cleaned_data

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arihanthtadanki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arihanthtadanki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
eng_data, esp_data = tuple(zip(*X))

cleaned_eng = sentence_preprocessor(eng_data, 'english')
cleaned_esp = sentence_preprocessor(esp_data, 'spanish')

In [5]:
len(cleaned_eng)

1000

In [6]:
# fasttext.util.download_model('en', if_exists='ignore')  # English

In [5]:
ft = fasttext.load_model('models/cc.en.300.bin')
ft.get_dimension()

300

In [11]:
fasttext.util.reduce_model(ft, 100)
ft.get_dimension()

100

In [6]:
ft2 = fasttext.load_model('models/cc.es.300.bin')
ft2.get_dimension()

300

In [12]:
fasttext.util.reduce_model(ft2, 100)
ft2.get_dimension()

100

In [10]:
from numpy.linalg import norm
from tqdm import tqdm

In [12]:
def cosine(A, B):
    return 5*(1+np.dot(A,B)/(norm(A)*norm(B)))/2

In [13]:
with open('./sts-2017-en-es/En_Es_STS/fasttext_scores.txt', 'w+') as f:
    for eng_sent, esp_sent in tqdm(zip(cleaned_eng, cleaned_esp)):
        # eng_sent = ' '.join(eng_sent)
        # esp_sent = ' '.join(esp_sent)
        eng_vect = [ft.get_sentence_vector(word) for word in eng_sent]
        esp_vect = [ft2.get_sentence_vector(word) for word in esp_sent]
        print(f"{eng_vect};{esp_vect}\n", file=f)

1000it [00:19, 51.67it/s]
